In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
iris = datasets.load_iris()
import random
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.models_fn.imputer_models import measure_val_error,XGBImputer,MySimpleImputer
from src.data_fn.data_process import simulate_missing_values,replace_infs
# from sklearn.metrics import mean_squared_error,f1_score

Importing C:\Users\johma\Desktop\main\4fun\AutoImputer\src\models_fn\imputer_models.py
Importing C:\Users\johma\Desktop\main\4fun\AutoImputer\src\data_fn\data_process.py


In [2]:
data = pd.read_csv('..\data\processed\iris_original.csv')
curr_df = simulate_missing_values(data,output_name=None,prop=0.3)
inf_df = curr_df.copy()
inf_df['target'] = inf_df['target'].map({0.0:'A',1.0:'B',2.0:'C'})
inf_df.iloc[0,2] = -np.inf
dtypes = ["numeric", "numeric", "numeric", "numeric", "categorical"]
cols = data.columns
dtype_list = dict(zip(cols, dtypes))
imp1 = MySimpleImputer(dtype_list=dtype_list)


In [3]:
pd.unique(curr_df['target'])

array([nan,  0.,  1.,  2.])

In [20]:
# import plotly.express as px
# def plot_nas(df: pd.DataFrame):
#     if df.isnull().sum().sum() != 0:
#         na_df = (df.isnull().sum() / len(df)) * 100      
#         na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
#         missing_data = pd.DataFrame({'Missing Ratio %' :na_df})
#         fig = px.bar(missing_data, x='Missing Ratio %',title=0.5,text_auto='d')
#         fig.update_layout(title_x=0.5)
#         fig.update_layout({
#         'plot_bgcolor': 'rgba(0, 0, 0, 0)',
#         'paper_bgcolor': 'rgba(0, 0, 0, 0)',
#         })
#         # fig.show()
#     return fig
# plot_nas(curr_df)

,Missing Ratio %
sepal length (cm),34.000000
petal width (cm),30.666667
sepal width (cm),30.000000
petal length (cm),28.000000
target,26.000000


In [10]:
errors = measure_val_error(curr_df, imp1, n_folds=5)
errors

{'sepal length (cm)': 0.4,
 'sepal width (cm)': 0.28,
 'petal length (cm)': 0.96,
 'petal width (cm)': 0.44,
 'target': 0.74}

In [34]:
# curr_df.loc[:,['sepal length (cm)']] = curr_df.loc[:,['sepal length (cm)']].values+1
# curr_df.loc[['sepal length (cm)','sepal width (cm)']]
curr_df.loc[:,['sepal length (cm)','sepal width (cm)']] = pd.DataFrame(
                curr_df[['sepal length (cm)','sepal width (cm)']]+1,
                index=curr_df.index,
            )
curr_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,14.1,12.5,1.4,0.2,NaN
1,NaN,12.0,NaN,NaN,NaN
2,13.7,12.2,NaN,0.2,0.0
3,13.6,12.1,1.5,0.2,0.0
4,14.0,12.6,NaN,0.2,0.0
...,...,...,...,...,...
145,15.7,12.0,NaN,2.3,2.0
146,15.3,11.5,5.0,1.9,2.0
147,NaN,12.0,5.2,2.0,2.0
148,NaN,NaN,NaN,2.3,2.0


In [25]:
# # imp1 = MySimpleImputer(dtype_list=dtype_list)
# imp1 = XGBImputer(dtype_list=dtype_list)

# curr_df = curr_df.dropna(axis=0, how="any").copy()
# errors = pd.DataFrame(columns=curr_df.columns)
# # n-fold cross validation
# for i in range(2):
#     fold_error = pd.DataFrame(0.0,columns=curr_df.columns,index=range(1))

#     nans = curr_df.mask(np.random.random(curr_df.shape) < 0.4)
#     res = imp1.impute(nans)

#     for curr_col in curr_df.columns:
#         if dtype_list[curr_col] == 'numeric':
#             fold_error.loc[0,curr_col] = mean_squared_error(y_true=curr_df[curr_col].values,y_pred=res[curr_col].values,squared=False)
#         elif dtype_list[curr_col] == 'categorical':
#             # F1 = 2 * (precision * recall) / (precision + recall)
#             if len(pd.unique(res[curr_col])) > 2:
#                 average='micro'
#             else:
#                 average = 'binary'
#             fold_error.loc[0,curr_col] = f1_score(y_true=curr_df[curr_col].values,y_pred=res[curr_col].values,average=average)


#     # curr_errors = pd.DataFrame(((res - curr_df) ** 2).mean(axis=0) ** (1 / 2)).T
#     errors = pd.concat([errors, fold_error], axis=0)

In [26]:
# curr_df[curr_col].values
# res[curr_col].values
# nans[curr_col]
# nans
# imp1.impute(nans)
# f1_score(y_true=curr_df[curr_col].values,y_pred=res[curr_col].values,average=average)

# curr_df.columns
# col = 'sepal length (cm)'
# mean_squared_error(y_true=curr_df[col].values,y_pred=res[col].values,squared=False)
# fold_error
# dtype_list
# errors

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,0.894874,0.403961,0.563456,0.214554,0.909091
0,0.429046,0.337993,0.505047,0.440637,0.772727


{'sepal length (cm)': 0.44,
 'sepal width (cm)': 0.31,
 'petal length (cm)': 1.29,
 'petal width (cm)': 0.51,
 'target': 0.65}

In [5]:
# # data[data.select_dtypes(include='number').columns]

# res

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.100000,3.5,1.400000,1.183784,A
1,4.900000,3.0,1.400000,1.183784,A
2,5.882243,3.2,3.844545,1.183784,C
3,4.600000,3.1,1.500000,1.183784,C
4,5.882243,3.6,3.844545,0.200000,A
...,...,...,...,...,...
145,6.700000,3.0,5.200000,2.300000,C
146,5.882243,2.5,3.844545,1.900000,C
147,6.500000,3.0,5.200000,2.000000,C
148,6.200000,3.4,3.844545,1.183784,C


In [5]:
dtypes = ["numeric", "numeric", "numeric", "numeric", "categorical"]
cols = data.columns
dtype_list = dict(zip(cols, dtypes))
# data, dtype_list = format_dtypes(data, dtypes=dtypes, cols=cols)
imp = XGBImputer(dtype_list=dtype_list, random_seed=42, verbose=0, cv=2)
res = imp.impute(curr_df)
